In [1]:
#Uniform-Random3
import numpy as np
import random
import time

# Số lượng biến và mệnh đề cho Uniform Random-3-SAT
num_vars = 10
num_clauses = 40

In [2]:
# Tạo bộ dữ liệu Uniform Random-3-SAT
def generate_uniform_random_3sat(num_vars, num_clauses):
    clauses = []
    for _ in range(num_clauses):
        # Chọn ngẫu nhiên 3 biến khác nhau
        clause = np.random.choice(num_vars, 3, replace=False)
        # Phủ định mỗi biến với xác suất 1/2
        clause = [var if np.random.rand() > 0.5 else -var for var in clause]
        clauses.append(clause)
    return clauses

# Tạo input cho Uniform Random-3-SAT
ur_3sat_instance = generate_uniform_random_3sat(num_vars, num_clauses)
print("Bài toán Uniform Random-3-SAT:")
print(ur_3sat_instance)


Bài toán Uniform Random-3-SAT:
[[-5, -2, -8], [-3, 0, -7], [-2, -8, -7], [0, 5, 3], [5, 2, -4], [-1, 5, -6], [-8, -3, -2], [-3, 1, 9], [7, 0, -4], [-7, 8, 0], [9, 0, 6], [5, 8, -4], [3, -4, 2], [2, -9, 8], [-2, 0, 6], [1, -3, 0], [9, 5, 3], [-8, -6, -7], [8, -3, 6], [-6, 3, 0], [-4, 2, 3], [3, -4, -2], [7, 4, -5], [4, -8, 3], [0, -6, -9], [-2, 0, 3], [9, 2, 0], [-8, 6, 7], [-6, 0, -1], [-4, 7, 0], [-7, -2, -8], [0, 4, -5], [9, 4, -5], [-4, 3, 8], [6, 8, 9], [-2, -7, -9], [5, 0, 7], [3, 1, -9], [8, 7, 4], [0, -6, -7]]


In [3]:
def generate_random_3sat_instance(num_vars, num_clauses, backbone_size):
    # Khởi tạo ví dụ với một danh sách rỗng các mệnh đề
    instance = []

    # Xác định tập backbone bằng cách chọn backbone_size biểu thức ngẫu nhiên
    backbone = set(random.sample(range(1, num_vars + 1), backbone_size))

    # Thêm phủ định vào tập backbone với xác suất 50%
    backbone = {lit if random.choice([True, False]) else -lit for lit in backbone}

    while len(instance) < num_clauses:
        # Tạo một mệnh đề với 3 biểu thức duy nhất
        clause = set()
        while len(clause) < 3:
            var = random.randint(1, num_vars)
            lit = var if random.choice([True, False]) else -var
            clause.add(lit)

        # Chuyển đổi mệnh đề thành tuple và thêm vào ví dụ
        instance.append(tuple(clause))

    # Đảm bảo các biểu thức trong backbone có mặt trong ví dụ
    for lit in backbone:
        if not any(lit in clause or -lit in clause for clause in instance):
            # Thêm một mệnh đề mới chứa biểu thức trong backbone
            clause = {lit, random.randint(1, num_vars), random.randint(1, num_vars)}
            instance.append(tuple(clause))

    return instance

backbone_size = 10  # Kích thước backbone
sat_instance = generate_random_3sat_instance(num_vars, num_clauses, backbone_size)
print("Bài toán Random-3-SAT với backbone:")
print(sat_instance)

Bài toán Random-3-SAT với backbone:
[(1, 10, 6), (-4, 5, -9), (9, 10, 7), (8, 9, 2), (-6, -10, 7), (8, 9, -9), (2, -9, 7), (2, 10, 6), (-7, -10, -1), (-8, 10, -6), (8, 9, -9), (9, 4, -10), (-5, -4, -10), (8, -7, -4), (8, 2, 6), (9, -10, 7), (1, 10, -7), (-8, 3, -2), (-7, -6, -1), (-8, 2, 7), (-4, -2, 7), (7, 5, -9), (3, -4, -9), (2, 10, -9), (-9, -2, 7), (1, 10, 3), (2, -4, -3), (9, 3, 6), (3, 4, 7), (-6, -5, 7), (10, 5, -2), (2, 4, -10), (-6, -5, 7), (8, 1, 2), (-8, 5, 7), (-7, 10, 5), (8, -7, 10), (-7, -4, 4), (9, -7, -9), (-7, 3, -3)]


In [4]:
def generate_alpha(num_vars):
    # Chọn ngẫu nhiên 3 biến khác nhau
    alpha_clause = np.random.choice(range(1, num_vars+1), 3, replace=False)
    # Phủ định mỗi biến với xác suất 1/2
    alpha_clause = [var if np.random.rand() > 0.5 else -var for var in alpha_clause]
    return alpha_clause

# Sử dụng hàm generate_alpha để tạo alpha
alpha = generate_alpha(num_vars)
print("Mệnh đề alpha ngẫu nhiên:")
print(alpha)


Mệnh đề alpha ngẫu nhiên:
[10, 5, -6]


### Uniform random 3sat

##### PL-Resolution

In [5]:
# Hàm kiểm tra xem một mệnh đề có phải là mệnh đề trống không
def is_empty_clause(clause):
    return len(clause) == 0

# Hàm kiểm tra xem hai mệnh đề có phải là mệnh đề đối nhau không
def is_complementary(clause1, clause2):
    for literal in clause1:
        if -literal in clause2:
            return True
    return False

# Hàm PL_Resolve để tạo ra các mệnh đề mới từ hai mệnh đề đầu vào
def PL_Resolve(clause1, clause2):
    resolved_clause = []
    for literal in clause1:
        if -literal not in clause2:
            resolved_clause.append(literal)
    return resolved_clause

# Hàm kiểm tra xem mệnh đề mới đã tồn tại trong tập mệnh đề hay chưa
def is_clause_in_set(clause, clause_set):
    for c in clause_set:
        if set(c) == set(clause):
            return True
    return False

# Thuật toán PL_Resolution
def PL_Resolution(clauses, alpha):
    # Thêm phủ định của alpha vào KB
    clauses_with_alpha = clauses + [[-lit for lit in alpha]]
    new_clauses = set(tuple(clause) for clause in clauses_with_alpha)
    while True:
        n = len(new_clauses)
        pairs = [(clause1, clause2) for clause1 in new_clauses for clause2 
                 in new_clauses if clause1 != clause2]
        for (clause1, clause2) in pairs:
            if is_complementary(clause1, clause2):
                resolved_clause = PL_Resolve(clause1, clause2)
                if is_empty_clause(resolved_clause):
                    return True
                if not is_clause_in_set(resolved_clause, new_clauses):
                    new_clauses.add(tuple(resolved_clause))
        if len(new_clauses) == n:
            return False

In [6]:
start_time = time.time()
# Kiểm tra tìm kiếm phép phân giải với KB và alpha
result = PL_Resolution(ur_3sat_instance, alpha)

end_time = time.time()
execution_time = end_time - start_time

if result:
    print("Có phân giải hợp lý với KB và alpha.")
else:
    print("Không có phân giải hợp lý với KB và alpha.")

print(f"Thời gian thực thi: {execution_time} giây.")

Có phân giải hợp lý với KB và alpha.
Thời gian thực thi: 0.0019981861114501953 giây.


##### DPLL

In [7]:
import random
import numpy as np

def dpll(clauses, assignment=[]):
    # Kiểm tra xem có mệnh đề rỗng nào không
    if [] in clauses:
        return False
    # Kiểm tra xem tất cả mệnh đề đã được thỏa mãn chưa
    if not clauses:
        return assignment
    # Chọn một biến ngẫu nhiên từ mệnh đề đầu tiên
    for clause in clauses:
        if clause:
            var = random.choice(clause)
            break
    # Tạo danh sách các mệnh đề mới sau khi gán giá trị cho biến
    def new_clauses(clauses, var):
        # Loại bỏ mệnh đề nếu chứa biến với giá trị đúng
        # Loại bỏ biến đối lập nếu mệnh đề chứa biến đối lập
        new_clauses = []
        for clause in clauses:
            if var in clause:
                continue
            new_clause = [x for x in clause if x != -var]
            new_clauses.append(new_clause)
        return new_clauses
    # Thử gán True cho biến và gọi đệ quy DPLL
    if dpll(new_clauses(clauses, var), assignment + [var]):
        return assignment + [var]
    # Thử gán False cho biến và gọi đệ quy DPLL
    if dpll(new_clauses(clauses, -var), assignment + [-var]):
        return assignment + [-var]
    # Nếu cả hai trường hợp đều không thỏa mãn, trả về False
    return False

In [8]:
start_time = time.time()
# Kiểm tra xem alpha có thể được suy ra từ ur_3sat_instance không
dpll_solution = dpll(ur_3sat_instance + [alpha])
end_time = time.time()
execution_time = end_time - start_time

if dpll_solution:
    print("Tìm được lời giải thỏa mãn KB và alpha:")
    print(dpll_solution)
else:
    print("Không tìm được lời giải thỏa mãn KB và alpha.")
print(f"Thời gian thực thi: {execution_time} giây.")

Tìm được lời giải thỏa mãn KB và alpha:
[-2]
Thời gian thực thi: 0.0 giây.


##### WalkSAT

In [9]:
def walksat(clauses, alpha, max_flips=10000, p=0.5):
    # Thêm phủ định của alpha vào KB
    clauses_with_alpha = clauses + [[-lit for lit in alpha]]

    # Khởi tạo một lời giải ngẫu nhiên
    n_vars = max(abs(var) for clause in clauses_with_alpha for var in clause)
    assignment = [random.choice([-1, 1]) for _ in range(n_vars)]
    for _ in range(max_flips):
        # Tìm mệnh đề không thỏa mãn
        unsatisfied = [clause for clause in clauses_with_alpha if not any(assignment[abs(var) - 1] == var // abs(var) for var in clause)]

        # Nếu không còn mệnh đề không thỏa mãn, trả về lời giải
        if not unsatisfied:
            return assignment

        # Chọn một mệnh đề không thỏa mãn
        clause = random.choice(unsatisfied)

        # Với xác suất p, chọn một biến ngẫu nhiên trong mệnh đề để đảo giá trị
        # Với xác suất 1-p, chọn biến làm giảm nhiều nhất số mệnh đề không thỏa mãn
        if random.random() < p:
            var = random.choice(clause)
        else:
            # Tạo danh sách các cặp (bool, var) cho mỗi biến trong mệnh đề
            bool_var_pairs = [(assignment[abs(var) - 1] != var // abs(var), var) for var in clause]

            # Sắp xếp danh sách theo thứ tự tăng dần của giá trị bool
            bool_var_pairs.sort()

            # Chọn biến đầu tiên (làm giảm nhiều nhất số mệnh đề không thỏa mãn)
            var = bool_var_pairs[0][1]

        # Đảo giá trị của biến đã chọn
        assignment[abs(var) - 1] *= -1

    # Nếu không tìm được lời giải sau số lần đảo giá trị tối đa, trả về None
    return None

In [10]:
start_time = time.time()
# Giải bài toán Uniform Random-3-SAT bằng WalkSAT với alpha
solution = walksat(ur_3sat_instance, alpha)

end_time = time.time()
execution_time = end_time - start_time

if solution:
    print("\nTìm được lời giải thỏa mãn KB và alpha:")
    solution_str = ['Yes' if value == 1 else 'No' for value in solution]
    print(solution_str)
else:
    print("\nKhông tìm được lời giải thỏa mãn KB và alpha.")
    
print(f"Thời gian thực thi: {execution_time} giây.")

C:\Users\FLOWER\AppData\Local\Temp\ipykernel_38464\3491197931.py:10: RuntimeWarning: divide by zero encountered in scalar floor_divide
  unsatisfied = [clause for clause in clauses_with_alpha if not any(assignment[abs(var) - 1] == var // abs(var) for var in clause)]
C:\Users\FLOWER\AppData\Local\Temp\ipykernel_38464\3491197931.py:25: RuntimeWarning: divide by zero encountered in scalar floor_divide
  bool_var_pairs = [(assignment[abs(var) - 1] != var // abs(var), var) for var in clause]



Không tìm được lời giải thỏa mãn KB và alpha.
Thời gian thực thi: 0.6106839179992676 giây.


### Random 3sat backbone

##### PL-Resolution

In [11]:
start_time = time.time()
result = PL_Resolution(sat_instance, alpha)

end_time = time.time()
execution_time = end_time - start_time

if result:
    print("Có phân giải hợp lý với KB và alpha.")
else:
    print("Không có phân giải hợp lý với KB và alpha.")
print(f"Thời gian thực thi: {execution_time} giây.")

Có phân giải hợp lý với KB và alpha.
Thời gian thực thi: 0.04857897758483887 giây.


##### DPLL

In [12]:
start_time = time.time()
dpll_solution = dpll(sat_instance + [alpha])

end_time = time.time()
execution_time = end_time - start_time

if dpll_solution:
    print("Tìm được lời giải thỏa mãn KB và alpha:")
    print(dpll_solution)
else:
    print("Không tìm được lời giải thỏa mãn KB và alpha.")
print(f"Thời gian thực thi: {execution_time} giây.")

Tìm được lời giải thỏa mãn KB và alpha:
[10]
Thời gian thực thi: 0.0 giây.


#### WalkSAT

In [13]:
start_time = time.time()

solution = walksat(sat_instance, alpha)

end_time = time.time()
execution_time = end_time - start_time

if solution:
    print("\nTìm được lời giải thỏa mãn KB và alpha:")
    solution_str = ['Yes' if value == 1 else 'No' for value in solution]
    print(solution_str)
else:
    print("\nKhông tìm được lời giải thỏa mãn KB và alpha.")
print(f"Thời gian thực thi: {execution_time} giây.")


Tìm được lời giải thỏa mãn KB và alpha:
['Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No']
Thời gian thực thi: 0.003997325897216797 giây.
